<a href="https://colab.research.google.com/github/IOS-Silva/projeto-dio-assistente-virtual-por-voz/blob/main/projeto_dio_assistente_virtual_por_voz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎙 Assistente de Voz com IA (100% Gratuito)

Este projeto implementa um assistente de voz utilizando tecnologias gratuitas.

Fluxo do sistema:

🎤 O usuário fala pelo microfone  
📁 O áudio é gravado e salvo  
📝 O modelo Whisper converte o áudio em texto  
🤖 Um modelo de linguagem (LLaMA 3 via Groq) gera a resposta  
🔊 A resposta é convertida em voz com gTTS  
🎧 O usuário escuta a resposta automaticamente  

O objetivo é demonstrar a integração entre captura de áudio, processamento de linguagem natural e síntese de voz em um único fluxo automatizado.

In [ ]:
!pip install -q groq
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.3 MB/s eta 0:00:00


# 1. Gravação de Áudio Com Python

In [ ]:
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  display(Javascript(RECORD))
  js_result = output.eval_js('record(%s)' % (sec * 1000))
  audio = b64decode(js_result.split(',')[1])
  file_name = 'request_audio.wav'
  with open(file_name, 'wb') as f:
    f.write(audio)
  return f'/content/{file_name}'

print("Ouvindo...")
record_file = record(5)
display(Audio(record_file))

Ouvindo...


<IPython.core.display.Javascript object>

# 2. Reconhecimento de Fala com Whisper

In [ ]:
import whisper

language = "pt"

model = whisper.load_model("base")
result = model.transcribe(record_file, fp16=False, language=language)

transcription = result["text"]

print("📝 Você disse:")
print(transcription)

📝 Você disse:
 um, dois, três gravando, está me ouvindo


# 3. Integração com a API do Groq

In [ ]:
from groq import Groq

client = Groq(api_key="COLAR-KEY-AQUI")

response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "user", "content": transcription}
    ],
)

ai_response = response.choices[0].message.content

print("🤖 Resposta da IA:")
print(ai_response)

🤖 Resposta da IA:
Não sou um ser humano e não tenho ouvidos físicos, mas posso ouvir o que você está dizendo através do texto. Sei que você está tentando falar em português e referindo-se a uma frase famosa, mas não sei ao certo o que você está procurando.

Se você estiver procurando discutir ou saber mais sobre algum assunto, estou aqui para ajudar. Se estiver apenas divertindo-se com a música, posso tentar ajudar com algo relacionado à música. Qual é o seu objetivo?


# 4. Sintetizando a Resposta do Groq Como Voz

In [ ]:
from gtts import gTTS
from IPython.display import Audio

tts = gTTS(text=ai_response, lang="pt", slow=False)

response_audio = "/content/response_audio.mp3"
tts.save(response_audio)

print("🔊 Reproduzindo resposta...")
Audio(response_audio, autoplay=True)

🔊 Reproduzindo resposta...
